### Instrucciones
1. Extraer los resúmenes de artículos de wikipedia que contengan la palabra 'astro' (limitar a máximo 20)
2. Crear una variable con 0 o 1 que indique si el resumen contiene alguna de las palabras 'física' o 'ciencia'.
3. Crear dataframe con columnas: pageid, titulo, resumen, contiene_palabra

Con esto podemos tener una idea (burda) de si el contenido es científico. Pueden elegir cualquier otra palabra para seleccionar los artículos y para la búsqueda dentro del resumen (o todo el texto del artículo).

(utilizar la API rest para obtener la lista de páginas)

In [16]:
import pandas as pd
import requests

### **1. Buscamos los artículos de Wikipedia que contengan la palabra "astro"**

In [17]:
buscar_en_titulo = 'astro'
#Realizamos la búsqueda en wikipedia
endpoint = 'https://es.wikipedia.org/w/rest.php/v1/search/title'
params = {
            'q' : buscar_en_titulo,
            'limit': 20 #Lo limitamos a máximo 20
        }

request_wiki = requests.get(endpoint, params=params)
#Verificamos el estatus
request_wiki.status_code

200

In [18]:
#Mostramos los datos de los artículos encontrados
df = pd.json_normalize(request_wiki.json()['pages'])
df

,id,key,title,excerpt,matched_title,description,thumbnail,thumbnail.mimetype,thumbnail.size,thumbnail.width,thumbnail.height,thumbnail.duration,thumbnail.url
0,59546,Objeto_astronómico,Objeto astronómico,Astro,Astro,"cuerpo físico de tamaño, masa o función astron...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,261,Astronomía,Astronomía,Astronomía,None,ciencia que estudia los cuerpos celestes del u...,NaN,image/jpeg,NaN,60.0,60.0,NaN,//upload.wikimedia.org/wikipedia/commons/thumb...
2,266,Astrología,Astrología,Astrología,None,forma de adivinación imaginaria basada en el e...,NaN,image/jpeg,NaN,60.0,78.0,NaN,//upload.wikimedia.org/wikipedia/commons/thumb...
3,257842,Astrología_china,Astrología china,Astrología china,None,horóscopo chino,NaN,image/png,NaN,60.0,60.0,NaN,//upload.wikimedia.org/wikipedia/commons/thumb...
4,9149789,Astronomía_maya,Astronomía maya,Astronomía maya,None,dioses de esa epoca,NaN,image/jpeg,NaN,60.0,87.0,NaN,//upload.wikimedia.org/wikipedia/commons/thumb...
5,15216,Astronáutica,Astronáutica,Astronáutica,None,teoría y práctica de la navegación fuera de la...,NaN,image/jpeg,NaN,60.0,71.0,NaN,//upload.wikimedia.org/wikipedia/commons/thumb...
6,6007265,Astrobiología,Astrobiología,Astrobiología,None,disciplina que estudia la existencia de vida f...,NaN,image/gif,NaN,60.0,34.0,NaN,//upload.wikimedia.org/wikipedia/commons/thumb...
7,7411986,Astro_(grupo_musical),Astro (grupo musical),Astro (grupo musical),None,grupo musical masculino surcoreano,NaN,image/png,NaN,60.0,28.0,NaN,//upload.wikimedia.org/wikipedia/commons/thumb...
8,3323,Astronauta,Astronauta,Astronauta,None,miembro de la tripulación o personal de una na...,NaN,image/jpeg,NaN,60.0,60.0,NaN,//upload.wikimedia.org/wikipedia/commons/thumb...
9,1779766,Astronomía_en_Chile,Astronomía en Chile,Astronomía en Chile,None,None,NaN,image/jpeg,NaN,60.0,54.0,NaN,//upload.wikimedia.org/wikipedia/commons/thumb...


In [25]:
#Extraemos los resúmenes
titles_list = df['title'].to_list()
pages_list = df['id'].to_list()
extracts_list = []
extracts_dict = {}

for page, title in zip(pages_list, titles_list):
    url = f'https://es.wikipedia.org/w/api.php?action=query&format=json&pageids={page}&prop=extracts&explaintext=True&exintro=True,'
    request_wiki = requests.get(url)
    request_wiki.status_code
    request_wiki.json()['query']['pages']
    data = pd.json_normalize(request_wiki.json()['query']['pages'])
    extract = data.iloc[:, 3][0]
    extracts_list.append(extract)
extracts_dict = { 'title' : titles_list, 'page_id' : pages_list, 'extract' : extracts_list}
extracts_df = pd.DataFrame(extracts_dict)
extracts_df

,title,page_id,extract
0,Objeto astronómico,59546,Un objeto astronómico es una entidad física si...
1,Astronomía,261,La astronomía (del griego άστρον [ástron] 'est...
2,Astrología,266,"La astrología, en su acepción más amplia, es u..."
3,Astrología china,257842,La astrología china es la astrología del calen...
4,Astronomía maya,9149789,La astronomía maya forma parte de una tradició...
5,Astronáutica,15216,La astronáutica es la teoría y práctica de la ...
6,Astrobiología,6007265,La astrobiología es la rama de la biología y l...
7,Astro (grupo musical),7411986,"Astro (en hangul, 아스트로) (estilizado como ASTRO..."
8,Astronauta,3323,Un astronauta[1]​ o cosmonauta[2]​ es una pers...
9,Astronomía en Chile,1779766,La astronomía en Chile ha sido desarrollada en...


### **2. Crear una variable con 0 o 1 que indique si el resumen contiene alguna de las palabras 'física' o 'ciencia'.**

In [26]:
verif_palabras = []

#Con un ciclo for recorremos todo el dataframe y fila por fila verifica si encuentra alguna de las palabras solicitadas
for index, row in extracts_df.iterrows():
    if 'ciencia' in row['extract'] or 'física' in row['extract']:
        #Si encuentra alguna de estas palabras agrega un uno a la lista 
        verif_palabras.append(1)
    else: 
        #De lo contrario agrega un cero a la lista
        verif_palabras.append(0)
        
verif_palabras 

[1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0]

### **3. Crear dataframe con columnas: pageid, titulo, resumen, contiene_palabra**

In [27]:
#Los resultados obtenidos anteriormente los convertimos a un DataFrame
verif_palabras = pd.Series(verif_palabras)

#Creamos el DataFrame con el nombre de las columnas solicitadas
extracts_df = pd.concat([extracts_df, verif_palabras.rename('contiene_palabra')],axis=1)
extracts_df

,title,page_id,extract,contiene_palabra
0,Objeto astronómico,59546,Un objeto astronómico es una entidad física si...,1
1,Astronomía,261,La astronomía (del griego άστρον [ástron] 'est...,1
2,Astrología,266,"La astrología, en su acepción más amplia, es u...",1
3,Astrología china,257842,La astrología china es la astrología del calen...,0
4,Astronomía maya,9149789,La astronomía maya forma parte de una tradició...,0
5,Astronáutica,15216,La astronáutica es la teoría y práctica de la ...,1
6,Astrobiología,6007265,La astrobiología es la rama de la biología y l...,1
7,Astro (grupo musical),7411986,"Astro (en hangul, 아스트로) (estilizado como ASTRO...",0
8,Astronauta,3323,Un astronauta[1]​ o cosmonauta[2]​ es una pers...,0
9,Astronomía en Chile,1779766,La astronomía en Chile ha sido desarrollada en...,1
